In [33]:
import os
import sys
import re
import datetime
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os
import subprocess

import numpy as np
import pandas as pd
import sys

In [36]:
#defining cutoffenergy and kpoing spacing arrays 
cutoff_energies = [600]

kpoint_spacings = [0.04]

pressures = [0]

elements = ["La", "Ru", "Bi"]

#confirm choices
print("Cutoff energies: ", cutoff_energies)
print("Kpoint Spacings: ", kpoint_spacings)
print("Pressures: ", pressures)
print("Elements: ", elements)

storage_directory = "/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles" 
print("Storage directory: ", storage_directory)

#cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test_1_File" - 1 file
#cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test_10etc" - 100 files
#cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test_10_files" #- 10 files
cell_file_directory = "/home/gridsan/tmackey/Data/KISS_cell_files_First_Test" #- all files
print("Cell file directory: ", cell_file_directory)

#in the 10 file test, the compounds concerneded are :
#K3 In Se3 - not supposed to produce
#Nd5 (Mo3 O16) - not supposed to produce
#K3 (C O3) - not supposed to produce
#Cs In Cl3 - not supposed to produce
#Cs Cu Bi - not supposed to produce
#Ba (Si2 O5) - not supposed to produce
#Cu Br Se - not supposed to produce 
#Ce Cu5 Sn - not supposed to produce
#K6 Ge2 Te6 - not supposed to produce
#Os3 (C O)12 - not supposed to produce
#La In S - manually copied in, supposed to produce

CASTEP_tools_directory = "/home/gridsan/tmackey/MyTools/CASTEP_Scripts"
print("CASTEP tools directory: ", CASTEP_tools_directory)

all_steps_script = "/home/gridsan/tmackey/MyTools/CASTEP_Scripts/all_steps.sh"
print("All steps script: ", all_steps_script)

all_ternary_compounds_directory = "/home/gridsan/tmackey/Data/all_ternary_compounds.csv"
print("All ternary compounds directory: ", all_ternary_compounds_directory)


Cutoff energies:  [600]
Kpoint Spacings:  [0.04]
Pressures:  [0]
Elements:  ['La', 'Ru', 'Bi']
Storage directory:  /home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles
Cell file directory:  /home/gridsan/tmackey/Data/KISS_cell_files_First_Test
CASTEP tools directory:  /home/gridsan/tmackey/MyTools/CASTEP_Scripts
All steps script:  /home/gridsan/tmackey/MyTools/CASTEP_Scripts/all_steps.sh
All ternary compounds directory:  /home/gridsan/tmackey/Data/all_ternary_compounds.csv


In [40]:
# remove any old data from the storage directory

def remove_directory(directory_path):
    # Kill processes that are accessing .nfs files in the directory
    # try:
    #     result = subprocess.run(["lsof | grep .nfs | awk '{print $2}' | xargs kill -9"], shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    #     if result.returncode != 0:
    #         print(f"Failed to kill processes accessing .nfs files in {directory_path}: {result.stderr.decode('utf-8')}")
    # except Exception as e:
    #     print(f"Failed to kill processes accessing .nfs files in {directory_path}: {str(e)}")

    # Remove the directory
    try:
        result = subprocess.run(["rm", "-rf", directory_path], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            print(f"Successfully removed {directory_path}")
        else:
            print(f"Failed to remove {directory_path}: {result.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Failed to remove {directory_path}: {str(e)}")

remove_directory(storage_directory)

Successfully removed /home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles


In [41]:
#initialize the data with all_steps
#create new directory within the storage directory corresponding to that particular condition / parameter

def generate_command(all_steps_script, storage_directory, cutoff_energy, kpoint_spacing, pressure, cell_file_directory, all_ternary_compounds_directory, runtype, elements):

    command = [all_steps_script, "-d", storage_directory, "-e", str(cutoff_energy), "-k", str(kpoint_spacing), "-p", str(pressure), "-c", cell_file_directory, "-t", all_ternary_compounds_directory, "-r", runtype] + elements
    
    try:
        result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if result.returncode == 0:
            print(f"Command executed successfully:\n{result.stdout.decode('utf-8')}")
        else:
            print(f"Failed to execute command: {result.stderr.decode('utf-8')}")
    except Exception as e:
        print(f"Failed to execute command: {str(e)}")

#test the initialization step 
generate_command(all_steps_script,storage_directory, cutoff_energies[0], kpoint_spacings[0], pressures[0], cell_file_directory, all_ternary_compounds_directory, "init", elements)
#what should happen:
#1. the new directory within the storage directory should be created

Command executed successfully:
starting reading_inputs
finished reading_inputs
start defining functions
/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaRuBi-600-0.04-0/
Running initialization instructions



In [42]:
#import the decoration and initailization script and confirm your output directory
import decoration_and_initialization as deco

output_dir = storage_directory + "/" + elements[0] + elements[1] + elements[2] + "-" + str(cutoff_energies[0]) + "-" + str(kpoint_spacings[0]) + "-" + str(pressures[0]) 
#test
print(output_dir)
# "/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaPbBi-600-0.04-0"

/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaRuBi-600-0.04-0


In [43]:
#update the deco package with importlib
import importlib
importlib.reload(deco)

deco.edit_all_cell_files(cell_file_directory, pressures[0], kpoint_spacings[0], elements, all_ternary_compounds_directory, output_dir)

#what should happen:
#1. 2 synthetic cell files should be created from the cell file in the input directory and copied to the output directory
#they should have the correct pressure, kpoint spacing and element substitutions made. their cutoff energies should be set to whatever the cutoff_energy is provided as 
#Pressure - checked
#Kpoints - checked

#for the folder of incomplete files (100etc) it correctly identified and labeled the cell files 

criteria for permutations with self-to-self substitutions is:  True
criteria for permutations with a false mapping is:  False


0.175060784994789860.154220215352552980.16672455713789510.170892671066342480.137547759638763450.18339701285168460.162556443209447730.179228898923237230.158388329281000360.187565126780131990.133379645710316070.20840569642236890.212573810350816270.237582493921500520.045849253212921150.229246266064605770.195901354637026740.00.03751302535602640.250086835706842650.0250086835706842640.079194164640500180.095866620354289680.254254949635290050.066689822855158040.233414379993053140.0583535949982632860.050017367141368530.0041681139284473770.0125043417853421320.070857936783605410.020840569642236890.0291767974991316430.062521708926710660.26259117749218480.100034734282737060.20423758249392150.22507815213615840.033344911427579020.054185481069815910.191733240708579360.15005210142410560.07502605071205280.09169850642584230.0083362278568947550.041681139284473780.20006946856547410.216741924279263640.016672455713789510.12921153178186870.245918721778395270.25842306356373740.108370962139631820.11670718999652

In [44]:
#take in something like "Synth_20381_LaLaInBiSPb.cell" and return "20381"

def get_code_from_file_name(file_name):
    return file_name.split("_")[1]

codes = [get_code_from_file_name(file_name) for file_name in os.listdir(output_dir)]

#remove duplicates
codes = list(set(codes))

print(codes)

['150777', '412576', '62893', '54733', '11406', '55511', '411077', '101016', '13930', '80005', '157336', '250752', '150404', '101011', '424333', '370040', '423170', '173171', '106529', '420766', '142736', '183347', '12736', '410419', '7781', '154741', '55495', '156950', '192462', '65000', '67988', '20350', '230729', '63511', '189116', '83378', '99240', '54570', '12049', '431662', '183028', '12738', '182050', '83907', '174471', '190838', '5620', '244020', '168470', '192802', '8494', '419491', '185105', '154791', '67985', '410732', '168458', '412577', '163719', '106417', '55668']


In [45]:
import importlib
importlib.reload(deco)

deco.init_csv("checkpoint.csv", output_dir)

for code in codes:
    deco.write_csv("checkpoint.csv", all_ternary_compounds_directory, code, os.path.join(cell_file_directory, code + ".cell"), output_dir, elements)

#this should produce an output in the test folder that looks like this for test folder 1 and the 10 file test folder 
# code,structure type,permutation,completion,error,error_type
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Pb', 'S': 'Bi'}",yes,,
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Bi', 'S': 'Pb'}",yes,,

criteria for permutations with self-to-self substitutions is:  True
criteria for permutations with a false mapping is:  False
Error: code not found in all_ternaries.csv
Error: code not found in all_ternaries.csv
Error: code not found in all_ternaries.csv


In [46]:
generate_command(all_steps_script,storage_directory, cutoff_energies[0], kpoint_spacings[0], pressures[0], cell_file_directory, all_ternary_compounds_directory, "first_step_only", elements)

#tested with the 10 file folder 
#tested the different components by commenting out the unwanted stuff
#when focusing on the .param file it produced .param files that 
#(a) all lines unmodified except for the very end 
#(b) a line specifying the cutoff energy at the very end

#when tested on the .param file and the .sbatch file 
#(a) the .param file was modified as above
#(b) the sbatch file was modified correctly to make it runnable on supercloud 
#EXAMPLE: 
# #!/bin/bash
# #SBATCH -N 1
# #SBATCH -n 4
# #SBATCH --array=1
# #SBATCH --mem=4500
# #SBATCH --time=04:00:00
# #SBATCH --output="output/Synth_20381_LaLaInPbSBi_output.txt"
# #SBATCH --error="error/Synth_20381_LaLaInPbSBi_error.txt"

# echo "Date              = $(date)"
# echo "Hostname          = $(hostname -s)"
# echo "Working Directory = $(pwd)"

# module add mpi/openmpi-4.1.3

# export PATH=/home/gridsan/tmackey/support/CASTEP-21.11/bin/linux_x86_64_gfortran--mpi:$PATH

# export PATH=/home/gridsan/tmackey/support/CASTEP-21.11/bin/linux_x86_64_gfortran--serial:$PATH

# mpirun -np 4 castep.mpi Synth_20381_LaLaInPbSBi

#uncommented the sbatch submission line in all_steps and everthing looks like it's running smoothly!

##### TESTING WITH ALL CELL FILES (>300 decorated cell files)

Command executed successfully:
starting reading_inputs
finished reading_inputs
start defining functions
/home/gridsan/tmackey/MyTools/CASTEP_Scripts/TestFolder/SuperCloudTest1/outputfiles/LaRuBi-600-0.04-0/
Running first step only instructions
Running generate_params.py
['Synth_54570_GdLaSnBiCoRu.cell', 'Synth_370040_TbLaInBiCoRu.cell', 'output', 'Synth_410419_PtRuYbLaSnBi.cell', 'Synth_67985_PrLaIBiFeRu.cell', 'Synth_12738_PtRuLaLaSnBi.cell', 'Synth_156950_AuRuGdLaSbBi.cell', 'Synth_174471_ErLaSbBiPdRu.cell', 'Synth_83378_CeLaSbBiPdRu.cell', 'Synth_431662_HoLaInBiRhRu.cell', 'Synth_101011_CeLaSnBiCuRu.cell', 'Synth_7781_ErLaSnBiNiRu.cell', 'Synth_150777_PtRuCeLaSnBi.cell', 'Synth_423170_IrRuLaLaSbBi.cell', 'Synth_412576_PtRuDyLaInBi.cell', 'Synth_182050_EuLaSnBiCuRu.cell', 'Synth_173171_YbLaSbBiCdRu.cell', 'Synth_154741_ErLaTeBiCuRu.cell', 'Synth_192462_ErLaSnBiCuRu.cell', 'Synth_13930_LaLaSbBiPdRu.cell', 'Synth_189116_CeLaSnBiPdRu.cell', 'Synth_168470_AuRuErLaTeBi.cell', 'Synth_41107

In [31]:
#GO TO TERMINAL AND MANUALLY PUSH THE JOB ARRAY(S) TO QUEUE USING SBATCH 
#######
#######
#######
#######

In [47]:
import collect

import importlib
importlib.reload(collect)

collect.main(output_dir)

#tested on the 10-file folder
#this should update the checkpoint file and create a log file that updates the round column everytime its called

#after 1st push, checkpoint file looks good:
# code,structure type,permutation,completion,error,error_type
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Pb', 'S': 'Bi'}",incomplete,False,N/A
# 20381,In5Nd4S13,"{'La': 'La', 'In': 'Bi', 'S': 'Pb'}",incomplete,False,N/A

#log file also looks good:
# name,n-bands round,n-bands error,suggested extra bands
# Synth_20381_LaLaInPbSBi.cell,0,0,0
# Synth_20381_LaLaInBiSPb.cell,0,0,0

#after 2nd push, checkpoint file looks the same and log file is appropriately updated 
# Synth_20381_LaLaInPbSBi.cell,0,0,0
# Synth_20381_LaLaInBiSPb.cell,0,0,0
# Synth_20381_LaLaInPbSBi.cell,1,0,0
# Synth_20381_LaLaInBiSPb.cell,1,0,0

False
['Synth_54570_GdLaSnBiCoRu.cell', 'Synth_370040_TbLaInBiCoRu.cell', 'Synth_410419_PtRuYbLaSnBi.cell', 'Synth_67985_PrLaIBiFeRu.cell', 'Synth_12049_BiBiHoLaPdRu-out.cell', 'Synth_12738_PtRuLaLaSnBi.cell', 'Synth_156950_AuRuGdLaSbBi.cell', 'Synth_174471_ErLaSbBiPdRu.cell', 'Synth_83378_CeLaSbBiPdRu.cell', 'Synth_431662_HoLaInBiRhRu.cell', 'Synth_101011_CeLaSnBiCuRu.cell', 'Synth_7781_ErLaSnBiNiRu.cell', 'Synth_150777_PtRuCeLaSnBi.cell', 'Synth_423170_IrRuLaLaSbBi.cell', 'Synth_412576_PtRuDyLaInBi.cell', 'Synth_182050_EuLaSnBiCuRu.cell', 'Synth_173171_YbLaSbBiCdRu.cell', 'Synth_154741_ErLaTeBiCuRu.cell', 'Synth_192462_ErLaSnBiCuRu.cell', 'Synth_13930_LaLaSbBiPdRu.cell', 'Synth_189116_CeLaSnBiPdRu.cell', 'Synth_168470_AuRuErLaTeBi.cell', 'Synth_411077_PrLaInBiNiRu.cell', 'Synth_192802_BiBiTbLaFeRu.cell', 'Synth_8494_PrLaSbBiPdRu.cell', 'Synth_419491_AuRuPrLaSnBi.cell', 'Synth_55495_CeLaSnBiNiRu.cell', 'Synth_101016_LaLaInBiNiRu.cell', 'Synth_150404_PrLaSbBiNiRu.cell', 'Synth_185105_L